# Beers Pipeline


### Prepare imported data:
* Import and concatenate beer style dfs
* Remove `'Brewery_Num'` and `'Beer_Name`' as they are not necessary for analysis
* Convert `'ABV'` and `'Beer_Score'` to `float` and `'Num_Beer_Ratings'` to `int`


### Add the following columns:
* Add Total Number of Beer Reviews
* Standard Deviation of `'ABV'`
* Number / Counts of Beer_Style produced by Brewery
* Average `'Beer_Score'` by Brewery
* Average ABV of Beers
* Max Number of Beer Ratings by Brewery
* Highest/Lowest Average Score for any Style for a Brewery
* Highest/Lowest Score for any Beer for a Brewery
* Max/min/mean/count by Beer_Style Category

In [1]:
import os

import pandas as pd
import numpy as np
import seaborn as snsm
import matplotlib.pyplot as plt
import seaborn as sns

import warnings; warnings.simplefilter('ignore')

In [2]:
os.chdir('../../Beergression_Data_and_misc/Data/')

# Beers

### Import and concatenate Beer Style Dataframes

The below was used to open all files that were saved to the Data2/ folder. Columns were renamed to be consistent with the convention used in brews_df.

``` python
data_fols = os.listdir()

style_nums = []

for fol in data_fols:
    try:
        int(fol)
        style_nums.append(str(fol))
    except:
        pass

beers_df = pd.DataFrame(columns = ['beer_name','brewery_name','abv',
                             'ratings','score','brewery_nums'])

for num in style_nums:
    pickled = pd.read_pickle(num)
    beers_df = pd.concat([beers_df,pickled],sort=True)
    
beers_df.columns = ['ABV','Beer_Name','Brewery_Name',
                    'Brewery_Num','Num_Beer_Ratings',
                    'Beer_Score','Beer_Style']
    
pd.to_pickle(beers_df,'Beer_Data')

```

In [4]:
beers_df = pd.read_pickle('Beer_Data')

Drop the Brewery_Num column as those values are only necessary to populate the Brewery Scraper function and will not be used in our model.

### Drop unnecessary columns

In [5]:
beers_df.drop(['Brewery_Num','Beer_Name'],inplace=True,axis=1)

Remove extraneous characters from `'ABV'` and `'Num_Beer_Ratings'` columns. Convert `'ABV'` and `'Beer_Score'` to `float` and `'Num_Beer_Ratings'` to `int`.

### Convert numeric columns to float and int

In [6]:
beers_df['ABV'].replace(' ? ',np.nan,inplace=True)
beers_df.Num_Beer_Ratings = beers_df.Num_Beer_Ratings.str.replace(',','')

beers_df.ABV = beers_df.ABV.apply(float)
beers_df.Beer_Score = beers_df.Beer_Score.apply(float)
beers_df.Num_Beer_Ratings = beers_df.Num_Beer_Ratings.apply(int)

Convert/impute missing values within ABV to the mean of ABV for that Beer_Style.  
I decided to use the mean instead of the median due to the fact that the distribution of alcohol percentages by beer style are roughly normal but values peak around around certain values, typically integers. As it was possible for the median to fall within some of these peaks around integers, I decided that the mean is likely a more accurate representation of central tendency.

### Add Total Number of Beer Reviews

In [7]:
total_beer_ratings = beers_df.groupby('Brewery_Name').sum().Num_Beer_Ratings
total_beer_ratings = pd.DataFrame(total_beer_ratings)
total_beer_ratings.columns = ['Total_Beer_Ratings']

### Determine Standard Deviation of `'ABV'`

It is possible that having a beers of a wider range of ABV could be something that customers seek in a brewery and thus lead to a higher score.

In [8]:
ABV_std = beers_df.groupby('Brewery_Name').std().ABV
ABV_std = pd.DataFrame(ABV_std)
ABV_std.columns = ['ABV_std']

### Number / Counts of Beer_Style produced by Brewery

In [9]:
num_styles_df = pd.DataFrame(beers_df.groupby(['Brewery_Name','Beer_Style']).size())
num_styles_df.reset_index(inplace=True)
num_styles = num_styles_df.groupby('Brewery_Name').size()
num_styles = pd.DataFrame(num_styles,columns=['Num_Styles'])

### Average `'Beer_Score'` by Brewery

In [10]:
mean_beer_score = beers_df.groupby('Brewery_Name').mean().Beer_Score
mean_beer_score = pd.DataFrame(mean_beer_score)
mean_beer_score.columns = ['Mean_Beer_Score']

### Average ABV of Beers

In [11]:
mean_abv = beers_df.groupby('Brewery_Name').mean().ABV
mean_abv = pd.DataFrame(mean_abv)
mean_abv.columns = ['Mean_ABV']

### Max Number of Beer Ratings by Brewery

In [12]:
max_beer_ratings = beers_df.groupby('Brewery_Name').max().Num_Beer_Ratings
max_beer_ratings = pd.DataFrame(max_beer_ratings)
max_beer_ratings.columns = ['Max_Num_Beer_Ratings']

### Highest/Lowest Average Score for any Style for a Brewery

In [13]:
mean_beer_score_by_style_df = pd.DataFrame(beers_df.groupby(['Brewery_Name','Beer_Style']).mean())
mean_beer_score_by_style_df.reset_index(inplace=True)

max_of_mean_beer_score = mean_beer_score_by_style_df.groupby('Brewery_Name').max().Beer_Score
max_of_mean_beer_score = pd.DataFrame(max_of_mean_beer_score)
max_of_mean_beer_score.columns = ['Max_Mean_Beer_Score']

min_of_mean_beer_score = mean_beer_score_by_style_df.groupby('Brewery_Name').min().Beer_Score
min_of_mean_beer_score = pd.DataFrame(min_of_mean_beer_score)
min_of_mean_beer_score.columns = ['Min_Mean_Beer_Score']

### Highest/Lowest Score for any Beer for a Brewery

In [14]:
max_beer_score = beers_df.groupby(['Brewery_Name']).max().Beer_Score
max_beer_score = pd.DataFrame(max_beer_score)
max_beer_score.columns = ['Max_Beer_Score']

min_beer_score = beers_df.groupby(['Brewery_Name']).min().Beer_Score
min_beer_score = pd.DataFrame(min_beer_score)
min_beer_score.columns = ['Min_Beer_Score']

## Creating Beer_Style Categories

### Max/min/mean/count by Beer_Style Category

Create patterns to identify style categories.

In [15]:
bocks = 'Bock'
brown_ales = 'Brown|Dark Mild|Altbier'
dark_ales = 'Black|Dark Ale|Dubbel|Roggenbier|Scottish Ale|Winter'
dark_lagers = 'Red Lager|Dark Lager|Märzen|Rauch|Schwarz|Dunkel Lager|Vien'
hybrid_beers = 'Cream|Champ|Cali'
ipa = 'IPA'
pale_ale = 'Red Ale|Blonde Ale|American Pale Ale|Belgian Pale|\
                    Belgian Saison|Bitter|English Pale|French|Kölsch'
pilsner = 'Adj|Pils|American L|Malt|Dort|\
                    Pale L|Strong L|Helles|Keller'
porter = 'Porter'
specialty = 'Chile|Sahti|Field|Spice|Japan|Low|\
                    Pumpkin|Kvass|Rye|Gruit|Smoke Beer'
stout = 'Stout'
strong = 'Barley|Imperial Red|Strong Ale|Wheatwine|\
                Quad|Strong Dark|Strong Pale|Tripel|Old|Wee'
wheat = 'Wheat Ale|Witbier|Weisse|Dunkelweizen|Hefe|Kristal'
wild_sour = 'Brett|Wild|Faro|Lambic|Gue|Flanders|Leip'

Combine lists of patterns and style category names. They will be zipped and used to loop through the dataframe and assign the appropriate style category name.

In [16]:
style_patterns = [bocks,brown_ales,dark_ales,dark_lagers,
                  hybrid_beers,ipa,pale_ale,
                  pilsner,porter,specialty,stout,strong,
                  wheat,wild_sour]
style_categories = ['Bocks','Brown_Ales','Dark_Ales','Dark_Lagers',
                  'Hybrid_Beers','IPA','Pale_Ale',
                  'Pilsner','Porter','Specialty','Stout','Strong',
                  'Wheat','Wild_Sour']

In [17]:
beers_df['Style_Category'] = None 

for pattern, style in zip(style_patterns,style_categories):
    beers_df.loc[beers_df.Beer_Style.str.contains(pattern),
           'Style_Category'] = style

Max/Min/Mean/Count are all assigned by grouping by both the Brewery Name and the style category. The pivot table function is used so that each style category for each brewery is its own column/feature. 

In [18]:
max_by_style = beers_df.groupby(['Brewery_Name',
                'Style_Category']).max()
max_by_style.reset_index(inplace=True)
max_by_style = max_by_style.pivot('Brewery_Name','Style_Category','Beer_Score')
max_by_style.columns = ['Max_' + str(col) for col in max_by_style.columns]

In [19]:
min_by_style = beers_df.groupby(['Brewery_Name',
                'Style_Category']).min()
min_by_style.reset_index(inplace=True)
min_by_style = min_by_style.pivot('Brewery_Name','Style_Category','Beer_Score')
min_by_style.columns = ['Min_' + str(col) for col in min_by_style.columns]

In [20]:
mean_by_style = beers_df.groupby(['Brewery_Name',
                'Style_Category']).mean()
mean_by_style.reset_index(inplace=True)
mean_by_style = mean_by_style.pivot('Brewery_Name','Style_Category','Beer_Score')
mean_by_style.columns = ['Mean_' + str(col) for col in mean_by_style.columns]

In [21]:
count_by_style = beers_df.groupby(['Brewery_Name',
                'Style_Category']).size()

count_by_style = pd.DataFrame(count_by_style)
count_by_style.reset_index(inplace=True)
count_by_style = count_by_style.pivot('Brewery_Name','Style_Category',0)
count_by_style.columns = ['Count_' + str(col) for col in count_by_style.columns]

Note below the creation a combined IPAs and Pale Ales set of features.

In [22]:
ipa_pale_ale_pattern = 'Red Ale|Blonde Ale|American Pale Ale|Belgian Pale|\
                    Belgian Saison|Bitter|English Pale|French|Kölsch|IPA'
ipa_pale_ale_df = beers_df[beers_df.Beer_Style.str.contains(ipa_pale_ale_pattern)]

max_ipa_pale_ale = ipa_pale_ale_df.groupby('Brewery_Name').max().Beer_Score
max_ipa_pale_ale = pd.DataFrame(max_ipa_pale_ale)
max_ipa_pale_ale.columns = ['Max_IPA_Pale_Ale']

min_ipa_pale_ale = ipa_pale_ale_df.groupby('Brewery_Name').min().Beer_Score
min_ipa_pale_ale = pd.DataFrame(min_ipa_pale_ale)
min_ipa_pale_ale.columns = ['Min_IPA_Pale_Ale']

mean_ipa_pale_ale = ipa_pale_ale_df.groupby('Brewery_Name').mean().Beer_Score
mean_ipa_pale_ale = pd.DataFrame(mean_ipa_pale_ale)
mean_ipa_pale_ale.columns = ['Mean_IPA_Pale_Ale']

count_ipa_pale_ale = ipa_pale_ale_df.groupby('Brewery_Name').size()
count_ipa_pale_ale = pd.DataFrame(count_ipa_pale_ale)
count_ipa_pale_ale.columns = ['Count_IPA_Pale_Ale']

# Compiling Beer Data

In [23]:
derived_from_beers = [total_beer_ratings, ABV_std, num_styles, 
                      mean_beer_score, mean_abv, max_beer_ratings, 
                      max_of_mean_beer_score, min_of_mean_beer_score, 
                      max_beer_score, min_beer_score]

style_dfs = [max_by_style, min_by_style, 
             mean_by_style, count_by_style,
             max_ipa_pale_ale, min_ipa_pale_ale,
             mean_ipa_pale_ale, count_ipa_pale_ale]

In [24]:
df = pd.concat(derived_from_beers,axis=1,join='outer')
df.fillna(0,inplace=True)
pd.to_pickle(df,'Clean_Beer_Data')

df = pd.concat(style_dfs,axis=1,join='outer')
df.fillna(0,inplace=True)
pd.to_pickle(df,'Clean_Beer_Style_Data')